In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
import torch.nn as nn
import csv

# Load Datasets

In [2]:
SNOPES_LOC = "./Datasets/Snopes/snopes.tsv"
#consists of rumors analyzed on the Snopes website along with their credibility labels (true or false), 
#sets of reporting articles, and their respective web sources

POLITIFACT_LOC = "./Datasets/PolitiFact/politifact.tsv"

In [3]:
snopes_df = pd.read_csv(SNOPES_LOC, sep='\t', header=None)
snopes_df.columns = ['Credibility', 'Claim_Source', 'Claim', 'Article', 'Article_Source']

## Preliminary Analysis

In [4]:
snopes_df.head()

,Credibility,Claim_Source,Claim,Article,Article_Source
0,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,there are several holidays throughout that tim...,www.godlikeproductions.com
1,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,defenses against same photographs show images ...,www.sjpba.net
2,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,the best that life could think out we extended...,www.englisher.net
3,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,this entry november 19 2006 published 9 years ...,rss2.com
4,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,place he will not do either said snape the ord...,www.englisher.net


In [5]:
print("Number of articles = {}".format(snopes_df.shape[0]))
print("Number of claims = {}".format(snopes_df.Claim_Source.nunique()))
#print("Number of true claims = {}".format(snopes_df.Claim_Source.nunique()))
#print("Number of false claims = {}".format(snopes_df.Claim_Source.nunique()))

Number of articles = 29242
Number of claims = 4341


In [15]:
glove_data_file = "./Glove/glove.6B.100d.txt"

words = pd.read_csv(glove_data_file, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [16]:
def vec(w):
  return words.loc[w].as_matrix()